In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        file_name = os.path.join(dirname, filename)
        print( filename)
        
df = pd.read_csv(file_name)
df.head()





# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Regression With Fastai¶

Regression With Fast.ai consists:

Importing necessary libraries
Creating a TabularList
Initialising Neural Network
Training the model
Evaluating the model
A simple analysis on the predictions of the validation set
Predicting using the network

In [ ]:
import pandas as pd
import numpy as np
from fastai.tabular import *

In [ ]:
from fastai.basics import *
from fastai.gen_doc.nbdoc import *

In [ ]:
import fastai
from fastai.version import __version__
print(__version__)

In [ ]:
from fastai.vision import *

In [ ]:
from fastai.tabular import *

In [ ]:
df.drop(['dteday'],axis=1,inplace=True)

In [ ]:
dep_var = 'cnt'
cat_names = ['season', 'yr', 'mnth', 'holiday', 'weekday','workingday', 'weathersit']
cont_names = ['temp','hum','windspeed']
procs = [Categorify]

In [ ]:
from sklearn.model_selection import train_test_split
train,test = train_test_split(df ,train_size = 0.15 , random_state = 38)

In [ ]:
test_tab = (TabularList.from_df(test,cat_names=cat_names, cont_names=cont_names, procs=procs))

In [ ]:
# Train data bunch
data_train = (TabularList.from_df(df, cat_names=cat_names, cont_names=cont_names, procs=procs)
                        .split_by_rand_pct(valid_pct = 0.1, seed = 42)
                        .label_from_df(cols = dep_var,label_cls = FloatList, log = True )
                        .add_test(test_tab)
                        .databunch())

In [ ]:
#Display the data batch
data_train.show_batch(rows = 5)

In [ ]:
# Create deep learning model
learn = tabular_learner(data_train, layers=[1000,200,15], metrics=[rmse,r2_score])

# select the appropriate learning rate
learn.lr_find()

# we typically find the point where the slope is steepest
learn.recorder.plot()


In [ ]:
#Fitting data and training the network
learn.fit_one_cycle(15, max_lr =1e-01)

In [ ]:
learn.show_results()

In [ ]:
#Getting The Training And Validation Errors

tr = learn.validate(learn.data.train_dl)
va = learn.validate(learn.data.valid_dl)
print("The Metrics used In Evaluating The Network:", str(learn.metrics))

print("\nThe calculated RMSE & R-Squared For The Training Set :", tr[1:])
print("\nThe calculated RMSE & R-Squared For The Validation Set :", va[1:])

In [ ]:
#Plotting The losses for training and validation
learn.recorder.plot_losses()

In [ ]:
#Plotting the metrics of evaluation
learn.recorder.plot_metrics()